# Monte Carlo collaborative target tracking  

In [ ]:
import collaborative_bearing_estimation as cbe
import numpy as np

# %matplotlib notebook
%matplotlib Qt5
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
import datetime

## setting up the scenario


In [ ]:

seed = np.random.randint(0,200)
print (seed)


#seed = 112 is not great for CCE vs CI possibly due to no implementation of rejecting inconsistent stuff
# seed= 67 # is a good case 
# seed= 62 # great for CCE but CI performs worse than KF possibly due to no rejection of inconsistent stuff
# seed = 197 # not great for CCE vs CI and almost matching Kalman
# seed = 50
np.random.seed (seed)

#initialize figure
x_range_plot = np.array([[-17],[25]])
y_range_plot = np.array([[-25],[20]])









# agent 1 and 2 list of positions with initial position added (same starting position for both)



# agent 1 and 2 list of positions with initial position added (same starting position for both)
x1 = np.array([[-15],[0]])
x2 = np.array([[8],[15]])

# target
x = np.array([[10],[-12]])





s1 = .3 # forward speed
a1 = np.deg2rad(0) #steering angle
xh1 = np.array([[2],[-1]])

e1 = 6**2
P1 = e1 * np.eye(2)
P1_inv = 1/e1 *np.eye(2)

min_est_r1 = 2 # min est range
max_est_r1 = 70 # max est range
est_ang_e1 = np.deg2rad(12) # est angle error std 

s2 = .3
a2 = np.deg2rad(-90)
xh2 = xh1 #np.array([[0],[0]])

e2 = e1 # 100
P2 = e2 * np.eye(2)
P2_inv = 1/e2 *np.eye(2)

min_est_r2 = 2 # min est range
max_est_r2 = 70 # max est range
est_ang_e2 = np.deg2rad(10) # est angle error std 

dt = 1
T = 90 # simulation time

F_m = 1 # Frequency of measurements max =1 
F_c = 1 # Frequency of fusion max =1 



# #create multiple agents that estimate the target in two lists of collaborative and non-collaborative
# a_l_NC = []
# a_l_C = []

# a1_NC = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, None)
# a1_CCE = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, "CCE")
# a1_CI = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, "CI")
# a1_Kalman = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, "Kalman")

# a_l_NC.append(a1_NC)
# a_l_C.append(a1_CCE)
# a_l_C.append(a1_CI)
# a_l_C.append(a1_Kalman)

# a2_NC = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, None)
# a2_CCE = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, "CCE")
# a2_CI = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, "CI")
# a2_Kalman = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, "Kalman")

# a_l_NC.append(a2_NC)
# a_l_C.append(a2_CCE)
# a_l_C.append(a2_CI)
# a_l_C.append(a2_Kalman)





## Monte Carlo Run

In [ ]:
# Monte Carlo Similation of 100 repeats with only measurement noise changing
MC_e = []
MC_c = []
MC_seed = []

MC_CCE_f = []
MC_CI_f = []
MC_K_f = []
MC_NC_f = []
MC_i = []

T_MC = 1000
for k in range(T_MC):
    
    # generate random seed
    seed = np.random.randint(0,T_MC)
    MC_seed.append(seed)
    np.random.seed (seed)
    
    
    s1 = .3 # forward speed
    a1 = np.deg2rad(0) #steering angle
    
    std = np.abs(np.random.normal (10, 10))
    x_e = np.random.normal(0, std, size=(2,2))
    print ("shape xe ",np.shape(x_e) )
    xh1 = x + x_e[0:2, 0:1] 
    
    e1 = std**2
    P1 = e1 * np.eye(2)
    P1_inv = 1/e1 *np.eye(2)
    
    min_r_e = np.random.normal (2, 5, size=2)
    max_r_e = np.random.normal (60, 20, size=2)
    est_ang_err = np.random.normal (5, 5, size=2)
    
    min_est_r1 = np. maximum (0, min_r_e[0]) # min est range
    max_est_r1 = np. maximum (0, max_r_e[0]) # max est range
    est_ang_e1 = np.deg2rad(np.abs(est_ang_err[0])) # est angle error std 

    s2 = .3
    a2 = np.deg2rad(-90)
    xh2 = x + x_e[0:2, 1:2]  #np.array([[0],[0]])
    

    
    e2 = std**2 # 100
    P2 = e2 * np.eye(2)
    P2_inv = 1/e2 *np.eye(2)

    min_est_r2 = np. maximum (0, min_r_e[1]) # min est range
    max_est_r2 = np. maximum (0, max_r_e[1]) # max est range
    est_ang_e2 = np.deg2rad(np.abs(est_ang_err[1])) # est angle error std 

    dt = 1
    T = 90 # simulation time

    F_m = 1 # Frequency of measurements max =1 
    F_c = 1 # Frequency of fusion max =1 
    
    #initialise the agents
    #create multiple agents that estimate the target in two lists of collaborative and non-collaborative
    a_l_NC = []
    a_l_C = []

    a1_NC = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, None)
    a1_CCE = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, "CCE")
    a1_CI = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, "CI")
    a1_Kalman = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, "Kalman")

    a_l_NC.append(a1_NC)
    a_l_C.append(a1_CCE)
    a_l_C.append(a1_CI)
    a_l_C.append(a1_Kalman)

    a2_NC = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, None)
    a2_CCE = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, "CCE")
    a2_CI = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, "CI")
    a2_Kalman = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, "Kalman")

    a_l_NC.append(a2_NC)
    a_l_C.append(a2_CCE)
    a_l_C.append(a2_CI)
    a_l_C.append(a2_Kalman)

    
    
    
    
    # generate errors
    e_1 = np.random.normal(0, a_l_NC[0].th_e, size= int(T/dt)) # agent 1's entire measurement error
    e_2 = np.random.normal(0, a_l_NC[1].th_e, size= int(T/dt)) 
    
    
    for a in a_l_C + a_l_NC :
        a.est_err(x) # capture the first estimation error
        
    MC_i.append((a_l_NC[0].err_l[-1] +  a_l_NC[1].err_l[-1])/2) # average of initial errors of agents
    
        
    for t in range (int(T/dt)):
                
        for a in a_l_NC + a_l_C:
        
            if a.ID ==1:
                
                cm, Pm, Pm_inv, f = a.sense(x, e_1[t])
            
            else: 
                cm, Pm, Pm_inv, f = a.sense(x, e_2[t])
        
        
            a.estimate(cm, Pm, Pm_inv, f)
            a.est_err(x)
    
        for a in a_l_C + a_l_NC : # after everyone has sensed then we can allow information exchange and move before the next round
        
            for m in [k for k in a_l_C if k.ID != a.ID and k.fuse_method == a.fuse_method ]: #only looking in collaborative agents with matching method
            
                            
                a.fuse (m)
        
#             a.move(dt)
    
    e_l =[]
    c_l = []
    for a in a_l_C + a_l_NC:
        e_l.append(a.err_l)
        c_l.append(a.conf_l)
    
    MC_e.append(e_l)
    MC_c.append(c_l)
    
    MC_CCE_f.append(a_l_C[0].c)
    MC_CI_f.append(a_l_C[1].c)
    MC_K_f.append(a_l_C[2].c)
    MC_NC_f.append(a_l_NC[1].c)
    
    
    
    
    


In [ ]:
# DATA Prepration for plotting
E = []
E_f = []
print (np.shape(MC_e))
print (len(a_l_C + a_l_NC))
for k in range(len(a_l_C + a_l_NC)):
    e_k = [[MC_e[j][k][i] for j in range(T_MC)] for i in range(int(T/dt)) ]
    E.append(e_k)
    
    e_f = [MC_e[j][k][-1] for j in range(T_MC)]
    E_f.append(e_f)
print (np.shape(E))
print (np.shape(E_f))


Sum = np.sum(MC_i)
alphas =.5 +  MC_i/ Sum
print ("MC_i", MC_i)
print (np.shape(MC_CI_f))
MC_CCE_f = np.reshape(MC_CCE_f, (T_MC, 2)) 
MC_CI_f = np.reshape(MC_CI_f, (T_MC, 2)) 
MC_K_f = np.reshape(MC_K_f, (T_MC, 2)) 
MC_NC_f = np.reshape(MC_NC_f, (T_MC, 2)) 


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.title ( "Seeds used")
ax.plot(MC_seed)

print (" SEEDS ", MC_seed )

fig = plt.figure()
ax2 = fig.add_subplot(111, aspect='equal')
ax2.set_xlim(x_range_plot[0,0], x_range_plot[1,0])
ax2.set_ylim(y_range_plot[0,0], y_range_plot[1,0])
plt.title("Point Cloud", fontsize=12)
plt.xlabel("x (m)", fontsize=12)
plt.ylabel("y (m)", fontsize=12)

ax2.scatter(x1[0,0], x1[1,0], s = 300, marker=">", alpha=1, c = "r", label= "Agent 1")
ax2.scatter(x2[0,0], x2[1,0], s = 300, marker="v", alpha=1, c = "b", label= "Agent 2")

ax2.scatter(MC_CCE_f[:,0], MC_CCE_f[:,1], marker="o", alpha = alphas, c = "r")
ax2.scatter(MC_CI_f[:,0], MC_CI_f[:,1], marker="o", alpha = alphas, c = "b")
ax2.scatter(MC_K_f[:,0], MC_K_f[:,1], marker="o", alpha = alphas, c = "g")
ax2.scatter(MC_NC_f[:,0], MC_NC_f[:,1], marker="o", alpha = alphas, c = "k")


ax2.scatter(x[0,0], x[1,0], s = 300, marker="*", alpha=1, c = "g", label= "Target")

In [ ]:



for i in range(8):
    
    if i ==0:
        ls = "-"
        color = 'b'
        method = 'CCE'
        agent = 1
    if i ==1:
        ls = "-"
        color = 'r'
        method = 'CI'
        agent = 1
    if i ==2:
        ls = "-"
        color = 'g'
        method = 'Kalman'
        agent = 1
    if i==3:
        ls = "--"
        color = 'b'
        method = 'CCE'
        agent = 2
    if i ==4:
        ls = "--"
        color = 'r'
        method = 'CI'
        agent = 2
    if i ==5:
        ls = "--"
        color = 'g'
        method = 'Kalman'
        agent = 2
    if i ==6:
        ls = "-"
        color = 'k'
        method = 'None'
        agent = 1
    if i==7:
        ls = "--"
        color = 'k'
        method = 'None'
        agent = 2
        
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     plt.title ( "Estimation Error of agent {0}-Method {1}".format(agent, method))
#     ax.boxplot(E[i])
    
    fig2 = plt.figure()
    ax2 = fig2.add_subplot(111)
    plt.title ( "Final Estimation Error of agent {0}-Method {1}".format(agent, method))
    ax2.plot(E_f[i])
    
#     , label="Agent_{0}, Method_{1}".format(a.ID, method)
    
    
    
# ax2.legend()


# fig3 = plt.figure()
# ax3 = fig3.add_subplot(111)
# plt.title ( "Det of Covariance")
# ax3.plot(a.conf_l, color=color, ls=ls, label="Agent_{0}, Method_{1}".format(a.ID, a.fuse_method))
# ax3.legend()
plt.show()
